In [1]:
!pip install pandas
import pandas as pd

"""
TURNOVER = 22976730.44
EV = 89740.29
PnL = 207358.91
COMISSION = 10367.95
NET_PnL = 196990.96
ROI = NET_PnL / TURNOVER * 100 = 0.86% (2DP)
"""

You should consider upgrading via the 'pip install --upgrade pip' command.


'\nTURNOVER =\nEV =\nPnL =\nCOMISSION =\nNET_PnL = \nROI = PnL / TURNOVER * 100\n'

In [2]:
# create a pandas data frame using horses.csv
horses_df = pd.read_csv("./questions/horses.csv")

In [3]:
# Define constants from problem
BANKROLL = 10000
MIN_BET = 2

In [17]:
# Add stake and back/lay columns to csv
# Kelly_criterion = (BP - (1-P))/B

def kelly_criterion(row):
    # calculate the kelly criterion
    price = row["win_starting_price"]
    inv_fair_price = 1 / row["win_fair_price"]
    numerator = price * inv_fair_price - (1 - inv_fair_price)
    denominator = price
    return (numerator/denominator)

def back_or_lay(row):
    back = 1 
    if kelly_criterion(row) < 0:
            back = 0
    return back
    
horses_df["kelly_criterion"] = horses_df.apply(
    lambda row: kelly_criterion(row), axis=1)

horses_df["stake"] = horses_df.apply(
    lambda row: abs(kelly_criterion(row)) * BANKROLL, axis=1)

horses_df["side"] = horses_df.apply(
    lambda row: back_or_lay(row), axis=1)

In [21]:
# TURNOVER CALCULATION
# TURNOVER is total amount staked in a bettors session
# exculding all bets below the min bet

TURNOVER = horses_df[horses_df.stake >= 2].sum()['stake']
print(TURNOVER)

22976730.43939194


In [18]:
horses_df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,stake,side,kelly_criterion
0,1,4,1.7353,1.7098,1,3284.444775,1,0.328444
1,1,1,6.0313,6.0914,0,288.548476,1,0.028855
2,1,5,7.6923,7.5101,0,141.561506,1,0.014156
3,1,6,20.3325,20.4978,0,27.960164,1,0.002796
4,1,2,23.9991,23.4710,0,8.377673,1,0.000838
...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,17.469732,1,0.001747
86644,10000,5,29.0062,29.8636,0,21.442353,1,0.002144
86645,10000,3,50.8005,48.9715,0,3.332293,0,-0.000333
86646,10000,9,310.1959,305.6448,0,0.374550,0,-0.000037


In [27]:
# Expected Value
# EV = (odds - 1)*stake*p - stake*q
def expected_value(row):
    # a lay is just the inverse of a back
    # The winnings of a back are equivalent to the liabities of a
    # lay, hence only a sign change is required 
    side = row['side']
    stake = row['stake']
    p = 1/row['win_fair_price']
    odds = row['win_starting_price']
    if not side:
        side = -1
    EV = (odds -1) * stake * p - stake * (1 - p)
    return (side * EV)

horses_df["EV"] = horses_df.apply( 
    lambda row: expected_value(row), axis=1
)



NameError: name 'horse_df' is not defined

In [30]:
TOTAL_EV = horses_df["EV"].sum()
print(TOTAL_EV)

89740.29823879989


In [35]:
# PnL is how much money was actually made, stake excluded
def PnL(row):
    winner = row["winner"]
    side = row["side"]
    stake = row["stake"]
    odds = row["win_starting_price"]
    amount = stake
    if winner:
        amount *= (odds - 1)
    if not side and winner:
        amount *= -1
    if side and not winner:
        amount *= -1
    return amount

horses_df["PnL"] = horses_df.apply( 
    lambda row: PnL(row), axis=1
)

horses_df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,stake,side,kelly_criterion,EV,PnL
0,1,4,1.7353,1.7098,1,3284.444775,1,0.328444,-48.264474,2331.298901
1,1,1,6.0313,6.0914,0,288.548476,1,0.028855,2.875294,-288.548476
2,1,5,7.6923,7.5101,0,141.561506,1,0.014156,-3.353029,-141.561506
3,1,6,20.3325,20.4978,0,27.960164,1,0.002796,0.227312,-27.960164
4,1,2,23.9991,23.4710,0,8.377673,1,0.000838,-0.184351,-8.377673
...,...,...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,17.469732,1,0.001747,-0.784721,-17.469732
86644,10000,5,29.0062,29.8636,0,21.442353,1,0.002144,0.633819,-21.442353
86645,10000,3,50.8005,48.9715,0,3.332293,0,-0.000333,0.119974,3.332293
86646,10000,9,310.1959,305.6448,0,0.374550,0,-0.000037,0.005495,0.374550


In [36]:
TOTAL_PNL = horses_df["PnL"].sum()
print(TOTAL_PNL)

207358.9138674928


In [37]:
# Comission - 5% total PnL 
Comission = 0.05 * TOTAL_PNL
print(Comission)

10367.94569337464


In [38]:
# Net PnL - PnL net Commission
NET_PNL = 0.95 * TOTAL_PNL
print(NET_PNL)

196990.96817411817
